In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import os
import smtplib
from email.message import EmailMessage
import time

email_address = "evelina.taralunga12@gmail.com"   
pwd = "fbni adju lwpf dlju"                  
to_email = "evelina.taralunga12@gmail.com"


previous_price = None
alert_count = 0
max_alerts = 3
threshold_percent = 0.0000000001 


def send_email_alert(current_price):
    global alert_count
    if alert_count >= max_alerts:
        return
    msg = EmailMessage()
    msg['Subject'] = "Alertă Bitcoin, prețul a scăzut :)"
    msg['From'] = email_address
    msg['To'] = to_email
    msg.set_content(f"Bitcoin a scăzut sub pragul setat!\nPreț curent: ${current_price}")
    
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(email_address, pwd)  
        smtp.send_message(msg)
    
    alert_count += 1
    print(f"Alertă trimisă! ({alert_count}/{max_alerts})")
        

def automated_crypto_pull():
    global previous_price
    url = 'https://coinmarketcap.com/currencies/bitcoin/'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    crypto_name = soup.find('span', {'data-role': 'coin-name'}).contents[0].strip()
    crypto_price_text = soup.find('span', {'data-test': 'text-cdp-price-display'}).text
    current_price = float(crypto_price_text.replace('$','').replace(',',''))
    
    date_time = datetime.now()
    
    data_dict = {'Crypto Name': crypto_name,
                 'Price': current_price,
                 'TimeStamp': date_time}
    
    df = pd.DataFrame([data_dict])
    print(df)

   
    csv_path = '/home/evelina/Desktop/Alex the Analist/Phyton/Crypto web puller/Crypto_Automated_Pull.csv'
    if os.path.exists(csv_path):
        df.to_csv(csv_path, mode='a', header=False, index=False)
    else:
        df.to_csv(csv_path, index=False)

    
    if previous_price is not None:
        drop_percent = (previous_price - current_price) / previous_price
        if drop_percent >= threshold_percent:
            send_email_alert(current_price) 

    previous_price = current_price


while True:
    automated_crypto_pull()
    time.sleep(10)  
